# Ciclos del modelo

El modelo de semejanza recorre un orden determinado de instrucciones que parametrizamos desde el script de criterios de ejecución

Los ciclos le dan un orden al proceso, una estructura homogenea a cada tabla de datos y flexibilidad de métodos para generar los resultados.

* Breve resumen de los ciclos:
    + inicio_modelo
    + ciclo por corridas 
    + ciclo por métodos

El proceso comienza en el script `inicio_modelo`, donde tenemos parametrizadas las bases de datos `origen` y `destino`. llamamos la función `calcular` del script `coordinador_criterios` parametrizada como `calcular(origen, destino)`.

In [ ]:

import os

import coordinador_criterios as homologar

import lectura_datos.datos_ministerio as datos_min
import lectura_datos.vehiculos as vehiculos

import utils.set_up as set_up

# Iniciamos los archivos de entrada y salida
set_up.inicio()

# Cargamos base de datos

tablas_ministerio = datos_min.min_imp()
vehiculos_sap, registros_unicos = vehiculos.cargar_sap(tipo = True)

# # Cálculamos criterios

homologar.calcular(registros_unicos, tablas_ministerio) # (origen, destino)

# Finalizamos para organizar los archivos de entrada y salida en orden periodico
set_up.final()

El proceso continua con el ciclo de `corrida`, este ciclo se parametriza en el script `criterios_ejecucion` y es representado por cada elemento de la lista `col_index`.  
en este punto tenemos las siguientes instrucciones:

In [ ]:
def calcular( origen, destino):
    
    # Iniciamos el dataframe 'sin_coincidencia' objetivo del proceso
    sin_coincidencia = origen.copy()
    
    # Iniciamos el ciclo por corrida, dado los n elementos de la lista col_index
    for i, iteracion in enumerate(ce.col_index):
        
        # Inicializamos el objeto 'indice'
        indice = datos.Indice(iteracion, ce.criterios)
        
        # Mostramos corrida actual
        ut.save_log(f'## Corrida: {indice.corrida}')

        if i == 0:
            
            sin_coincidencia = sin_coincidencia[indice.origen_id + indice.estrictas + indice.flexibles + indice.conservar_origen]

            exacta = pd.DataFrame()
            homologaciones = pd.DataFrame()
        
        else:

            sin_coincidencia.rename(index = str, columns = {'LINEA': 'LINEA2'}, inplace = True)
            
        # Iniciamos el ciclo por métodos
        for criterio, valor in indice.criterios.itertuples(index=False):  
            {
                
            }
        

El proceso continua con el ciclo por métodos, donde se lleva el seguimiento de los dataframe `exacta`, `homologaciones` y `sin coincidencia`, que contienen el seguimiento de todos los registros de `origen` durante el proceso.  
Este proceso es fundamental para coordinas los filtros y las funciones aplicadas a los registros de `origen`, por tal razón se debe garatizar que se conserva la integridad de esta base de datos cada vez que modificamos o agregamos nuevos métodos.

In [ ]:
# Iniciamos el ciclo por métodos
for criterio, valor in indice.criterios.itertuples(index=False):  

            #Calculamos desagregar
                    
            if criterio == 'desagregar' and valor:
                ut.save_log('### Desagregando')
                sin_coincidencia = desagregar.desagregar(indice, 
                        sin_coincidencia)

            # Calculamos exactas
            
            if criterio == 'exactas' and valor:
                ut.save_log('### Coincidencias exactas')                
                if len(exacta) <= 0:
                    k = 1
                    exacta, sin_coincidencia = exactas.exactas(indice, 
                            sin_coincidencia, destino, k)
                else: # if len(exacta) > 0:
                    k += 1
                    temp = exacta
                    exacta, sin_coincidencia = exactas.exactas(indice, 
                            sin_coincidencia, destino, k)
                    exacta = pd.concat([temp, exacta])

                # Imprimir exactas

                name = 'exactas_corrida_' + str(i+1) + '.csv'
                exacta.to_csv(os.path.abspath(os.path.join(path, name)), sep = ';', 
                        index = False)
                ut.save_log(f'Archivos generado: {name}')
                
            # Calculamos semajanza
            
            if criterio == 'semejanza' and valor:
                ut.save_log('### Calculando semejanza') 
                if len(homologaciones)>0:
                    temp = homologaciones.copy()
                    homologaciones, sin_coincidencia = semejanza.semejanza(
                            indice, sin_coincidencia, destino)
                    
                    homologaciones = pd.concat([temp, homologaciones]) 
                else:
                    homologaciones, sin_coincidencia = semejanza.semejanza(
                            indice, sin_coincidencia, destino)  
                
            # Calculamos contenida

            if criterio == 'palabras_contenidas' and valor:
                ut.save_log('### Calculando palabras contenidas') 
                
                if len(homologaciones)>0:
                    temp = homologaciones.copy()
                    homologaciones, sin_coincidencia = (
                            contenida.palabra_contenida_sin_orden(indice, 
                                    sin_coincidencia, destino)
                            )
                    
                    homologaciones = pd.concat([temp, homologaciones], sort=False) 
                else:
                    homologaciones, sin_coincidencia = (
                            contenida.palabra_contenida_sin_orden(
                                    indice, sin_coincidencia, destino)
                            )
                
            # Calculamos desempate

            if criterio == 'desempate' and valor:
                for columna in indice.desempate_cols:
                    
                    # Cada columna acumula en homologaciones los resultados filtrados en homologaciones
                    # Es importante no perder ningún registro en origen_id durante este proceso
                    homologaciones = desempate.desempate(indice, homologaciones, columna)
                    duplicados = df[df[indice.origen_id].duplicated(keep = False)]
                    if duplicados.shape[0] > 0:
                        ut.save_log(f'Existen duplicados despues correr desempate por {columna}')
#                        import pdb; pdb.set_trace()
            
            # Coincidencias cruzadas
            
            if criterio == 'cruzar' and valor:
                
                cruzar.destino_origen(indice, destino)


Despues de finalizar el ciclo por métodos, el modelo genera los archivos de salida de las tablas `homologaciones` y `sin coincidencia`, además genera el informe o 'bitácora' del modelo.

In [ ]:
           
ut.save_log('## Generación de archivos')

# Imprimir homologaciones

name = 'homologaciones_corrida_' + str(i+1) + '.csv'
homologaciones.to_csv(os.path.abspath(os.path.join(path, name)), 
        sep = ';', index = False)
ut.save_log(f'Archivos generado: {name}')
ut.docs_save(name)

# Imprimir sin coincidencia

name = 'sin_coincidencia_corrida_' + str(i+1) + '.csv'
sin_coincidencia.to_csv(os.path.abspath(os.path.join(path, name)), 
        sep = ';', index = False)
ut.save_log(f'Archivos generado: {name}')
ut.docs_save(name)

# Generación informes

gi.generar_informe()

return homologaciones, sin_coincidencia

A continuación el procedimiento completo:

In [ ]:
import pandas as pd
import os

import objetos.datos as datos
import criterios_ejecucion as ce
import metodos.desagregar as desagregar
import metodos.exactas as exactas
import metodos.semejanza as semejanza
import metodos.contenida as contenida
import metodos.desempate as desempate
import metodos.cruzar as cruzar

import utils.utils as ut
import utils.generar_informe as gi
import metadata.directory as metadir

path = metadir.Path.CARPETA_RESULTADOS

def calcular( origen, destino):

    sin_coincidencia = origen.copy()
    
    for i, iteracion in enumerate(ce.col_index):
        
        indice = datos.Indice(iteracion, ce.criterios)
        
        # Mostramos corrida actual
        ut.save_log(f'## Corrida: {indice.corrida}')

        # Criterios de la   corrida

        # print('Criterios: exactas, semejanza, palabras_contenidas, desempate, cruzar, desagregar')
        # in_criterios = input('¿Cuáles métodos desea hacer?:')

        if i == 0:
            
            sin_coincidencia = sin_coincidencia[indice.origen_id + indice.estrictas + indice.flexibles + indice.conservar_origen]

            exacta = pd.DataFrame()
            homologaciones = pd.DataFrame()
        
        else:

            sin_coincidencia.rename(index = str, columns = {'LINEA': 'LINEA2'}, inplace = True)

        for criterio, valor in indice.criterios.itertuples(index=False):  

            #Calculamos desagregar
                    
            if criterio == 'desagregar' and valor:
                ut.save_log('### Desagregando')
                sin_coincidencia = desagregar.desagregar(indice, 
                        sin_coincidencia)

            # Calculamos exactas
            
            if criterio == 'exactas' and valor:
                ut.save_log('### Coincidencias exactas')                
                if len(exacta) <= 0:
                    k = 1
                    exacta, sin_coincidencia = exactas.exactas(indice, 
                            sin_coincidencia, destino, k)
                else: # if len(exacta) > 0:
                    k += 1
                    temp = exacta
                    exacta, sin_coincidencia = exactas.exactas(indice, 
                            sin_coincidencia, destino, k)
                    exacta = pd.concat([temp, exacta])

                # Imprimir exactas

                name = 'exactas_corrida_' + str(i+1) + '.csv'
                exacta.to_csv(os.path.abspath(os.path.join(path, name)), sep = ';', 
                        index = False)
                ut.save_log(f'Archivos generado: {name}')
                
            # Calculamos semajanza
            
            if criterio == 'semejanza' and valor:
                ut.save_log('### Calculando semejanza') 
                if len(homologaciones)>0:
                    temp = homologaciones.copy()
                    homologaciones, sin_coincidencia = semejanza.semejanza(
                            indice, sin_coincidencia, destino)
                    
                    homologaciones = pd.concat([temp, homologaciones]) 
                else:
                    homologaciones, sin_coincidencia = semejanza.semejanza(
                            indice, sin_coincidencia, destino)  
                
            # Calculamos contenida

            if criterio == 'palabras_contenidas' and valor:
                ut.save_log('### Calculando palabras contenidas') 
                
                if len(homologaciones)>0:
                    temp = homologaciones.copy()
                    homologaciones, sin_coincidencia = (
                            contenida.palabra_contenida_sin_orden(indice, 
                                    sin_coincidencia, destino)
                            )
                    
                    homologaciones = pd.concat([temp, homologaciones], sort=False) 
                else:
                    homologaciones, sin_coincidencia = (
                            contenida.palabra_contenida_sin_orden(
                                    indice, sin_coincidencia, destino)
                            )
                
            # Calculamos desempate

            if criterio == 'desempate' and valor:
                for columna in indice.desempate_cols:
                    
                    # Cada columna acumula en homologaciones los resultados filtrados en homologaciones
                    # Es importante no perder ningún registro en origen_id durante este proceso
                    homologaciones = desempate.desempate(indice, homologaciones, columna)
                    duplicados = df[df[indice.origen_id].duplicated(keep = False)]
                    if duplicados.shape[0] > 0:
                        ut.save_log(f'Existen duplicados despues correr desempate por {columna}')
#                        import pdb; pdb.set_trace()
            
            # Coincidencias cruzadas
            
            if criterio == 'cruzar' and valor:
                
                cruzar.destino_origen(indice, destino)

    ut.save_log('## Generación de archivos')

    # Imprimir homologaciones

    name = 'homologaciones_corrida_' + str(i+1) + '.csv'
    homologaciones.to_csv(os.path.abspath(os.path.join(path, name)), 
            sep = ';', index = False)
    ut.save_log(f'Archivos generado: {name}')
    ut.docs_save(name)

    # Imprimir sin coincidencia

    name = 'sin_coincidencia_corrida_' + str(i+1) + '.csv'
    sin_coincidencia.to_csv(os.path.abspath(os.path.join(path, name)), 
            sep = ';', index = False)
    ut.save_log(f'Archivos generado: {name}')
    ut.docs_save(name)
    
    # Generación informes

    gi.generar_informe()

    return homologaciones, sin_coincidencia
            